# Stochastic Simulation Assignment 3


### **Contributors**  
- **Maarten Stork**  
- **Paul Jungnickel**  
- **Lucas Keijzer**

### **Overview**  
This notebook contains the code and analysis for **Assignment 3 of Stochastic Simulation**. The code follows the order specified in the assignment guidelines and replicates the experiments conducted in the referenced paper. Each section corresponds to (a) key experiment(s).

In [ ]:
from CircleParticleSim import *
import numpy as np
import numpy.random as rand
import pandas as pd
import matplotlib.pyplot as plt

# 1)

In [ ]:
def run_experiment(max_particles, steps, num_runs=10):
    """
    Run the simulation for a range of particle numbers and record results.

    Parameters:
    - max_particles: Maximum number of particles to test.
    - steps: Number of simulation steps.
    - num_runs: Number of simulation runs to average results.

    Returns:
    - results: A list of dictionaries containing particle count, average internal count, min/max internal count, and energy.
    - examples: List of tuples containing particle count and their locations.
    - raw_data: Raw data collected for boxplot visualization.
    """
    results = []
    examples = []
    raw_data = []

    for num_particles in range(2, max_particles + 1):
        internal_counts = []
        energies = []
        example_locations = None

        for run in range(num_runs):
            sim = CircleParticleSim(
                N=num_particles,
                cooling_schedule=basic_cooling_schedule,
                step_size_schedule=random_step_size_schedule,
                steps=steps
            )
            sim.run_simulation(steps)

            # Calculate floating particles (not on the edge)
            floating_count = np.sum(np.linalg.norm(sim.particle_locations, axis=1) < 0.99)
            internal_counts.append(floating_count)
            energies.append(sim.E)

            # Save example locations from the first run
            if run == 0:
                example_locations = sim.particle_locations

            # Store raw data for boxplot
            raw_data.append({
                "Particles": num_particles,
                "Run": run,
                "Floating Count": floating_count,
                "Energy": sim.E
            })

        avg_internal_count = np.mean(internal_counts)
        min_internal_count = np.min(internal_counts)
        max_internal_count = np.max(internal_counts)
        avg_energy = np.mean(energies)

        results.append({
            "Particles": num_particles,
            "Internal Count (Avg)": avg_internal_count,
            "Internal Count (Min)": min_internal_count,
            "Internal Count (Max)": max_internal_count,
            "Total Energy (Avg)": avg_energy
        })

        examples.append((num_particles, example_locations))
        print(f"Completed simulation for {num_particles} particles (averaged over {num_runs} runs).")

    return results, examples, raw_data


def plot_results_and_examples(results, examples):
    """
    Plot the results table and one example configuration for each particle count in a combined figure.

    Parameters:
    - results: List of dictionaries containing particle count, average internal count, min/max internal count, and energy.
    - examples: List of tuples containing particle count and their locations.
    """
    results_df = pd.DataFrame(results)
    print("Summary Table:")
    print(results_df)

    # Plot particle configurations
    num_examples = len(examples)
    fig, axes = plt.subplots(nrows=(num_examples // 4) + 1, ncols=4, figsize=(15, 10))
    axes = axes.flatten()

    for idx, (num_particles, locations) in enumerate(examples):
        ax = axes[idx]
        thetas = np.linspace(0, 2 * np.pi, 100)
        ax.plot(np.cos(thetas), np.sin(thetas), linestyle=':', color='gray')
        ax.scatter(locations[:, 0], locations[:, 1])
        ax.set_title(f"{num_particles} Particles")
        ax.set_xlim([-1.1, 1.1])
        ax.set_ylim([-1.1, 1.1])
        ax.axis('off')

    # Turn off unused subplots
    for ax in axes[len(examples):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


def plot_boxplots(raw_data):
    """
    Create boxplots of floating counts by particle count.

    Parameters:
    - raw_data: List of dictionaries with raw data from the simulation.
    """
    df = pd.DataFrame(raw_data)
    particle_counts = sorted(df["Particles"].unique())

    plt.figure(figsize=(12, 6))
    plt.boxplot(
        [df[df["Particles"] == p]["Floating Count"].values for p in particle_counts],
        labels=particle_counts
    )
    plt.xlabel("Number of Particles")
    plt.ylabel("Floating Points")
    plt.title("Distribution of Floating Points for Each Particle Count")
    plt.xticks(range(1, len(particle_counts) + 1), particle_counts)
    plt.grid(True)
    plt.show()


def plot_min_max_avg(results):
    """
    Plot min, max, and average floating counts for each particle count.

    Parameters:
    - results: List of dictionaries containing results summary.
    """
    results_df = pd.DataFrame(results)

    plt.figure(figsize=(12, 6))
    plt.plot(results_df["Particles"], results_df["Internal Count (Avg)"], label="Average", marker="o")
    plt.plot(results_df["Particles"], results_df["Internal Count (Min)"], label="Min", marker="o")
    plt.plot(results_df["Particles"], results_df["Internal Count (Max)"], label="Max", marker="o")
    plt.xlabel("Number of Particles")
    plt.ylabel("Floating Points")
    plt.title("Min, Max, and Average Floating Points by Particle Count")
    plt.xticks(results_df["Particles"])
    plt.legend()
    plt.grid(True)
    plt.show()


if __name__ == '__main__':
    num_particles = 5
    max_particles = 20
    steps = 10000
    num_runs = 10

    results, examples, raw_data = run_experiment(max_particles, steps, num_runs)

    plot_results_and_examples(results, examples)

    plot_boxplots(raw_data)

    plot_min_max_avg(results)


# 2)

In [15]:
# Parameters used
num_particles = 12
steps = 10000
num_runs = 50

schedules = [
# log_cooling_schedule,
# basic_cooling_schedule,
paper_cooling_schedule,
exponential_cooling_schedule,
# linear_cooling_schedule,
# quadratic_cooling_schedule,
sigmoid_cooling_schedule,
inverse_sqrt_cooling_schedule,
cosine_annealing_cooling_schedule,
# stepwise_cooling_schedule,
]

In [ ]:
data = {}

# generate data
for schedule in schedules:
    print(f"Currently running for: {schedule.__name__}")
    mean_energy, std_energy, mean_temperatures = evaluate_multiple_runs(
        num_particles, cooling_schedule=schedule, steps=steps, num_runs=num_runs
    )
    data[schedule.__name__] = {
        "mean_energy": mean_energy,
        "std_energy": std_energy,
        "mean_temperatures": mean_temperatures
    }

# Save data
np.save('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), data)


In [ ]:
loaded_data = np.load('data/data-2-{}-{}.npy'.format(num_particles, len(schedules)), allow_pickle=True).item()

fig, axs = plt.subplots(2, 1, figsize=(10, 12), sharex=True)

for schedule_name, results in loaded_data.items():
    mean_energy = results["mean_energy"]
    std_energy = results["std_energy"]
    mean_temperatures = results["mean_temperatures"]
    print(f"min energy: {min(mean_energy)} for schedule: {schedule_name}")
    axs[0].plot(mean_energy, label=schedule_name)
    axs[0].fill_between(
        range(len(mean_energy)),
        mean_energy - std_energy,
        mean_energy + std_energy,
        alpha=0.3
    )
    axs[1].plot(mean_temperatures, label=schedule_name)

# Plot for energy
axs[0].set_ylabel("Energy")
axs[0].set_title("Mean Energy with Standard Deviation Over Time (Loaded Data)")
axs[0].set_xscale('log')
axs[0].set_yscale('log')
axs[0].legend()
axs[0].grid(True)

# Plot for temperature
axs[1].set_xlabel("Steps")
axs[1].set_ylabel("Temperature")
axs[1].set_title("Temperature Evolution Over Time (Loaded Data)")
axs[1].set_xscale('log')
# axs[1].legend()
axs[1].grid(True)

skip_first_steps = 0
plt.xlim(left=skip_first_steps)
plt.tight_layout()
plt.show()

# 3)

In [ ]:
rand.seed(42)
run_count = 100
num_scales = 50
num_particles = 50
scales = np.logspace(-2,1,num=num_scales, base=10)
print(scales)
arrs = np.zeros([num_scales, run_count]) 

for i, scale in enumerate(scales):
    num_steps = int(100000 * scale)
    for j in range(run_count):
        sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                    cooling_schedule = paper_cooling_schedule,
                    step_size_schedule = sqrt_step_size_schedule,
                    random_step_likelihood=0.2,
                    extra_args = {'cooling_schedule_scaling' : scale} 
                    )
        sim.run_simulation(num_steps)
        arrs[i,j] = sim.E

    print('.', end='')

    
np.save('data/data-3-{}-{}.npy'.format(num_particles, num_scales), arrs)

In [ ]:

arrs = np.load('data/data-3-50-50.npy')
arrs = arrs[:]
scales = np.logspace(-2,1,num=num_scales, base=10)
scales1 = 100*scales[:]
print(arrs.shape)
mean = np.mean(arrs, axis=1)
min_energy = np.min(arrs)
plt.plot(scales1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
plt.fill_between(scales1, np.percentile(arrs, 0, axis=1), np.percentile(arrs, 99, axis=1), alpha=0.3)
plt.plot(scales1, min_energy*np.ones_like(scales1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('chain length scaling')
plt.ylabel('E')
plt.xscale('log')
plt.ylim([2900, 2950])
plt.legend()

# 4)

In [ ]:
rand.seed(42)
run_count = 100
num_probs = 50
num_particles = 16
probs = np.linspace(0,1,num_probs)
arrs = np.zeros([num_probs, run_count]) 

for i, p in enumerate(probs):
    for j in range(run_count):
        sim  = CircleParticleSim(num_particles, steps=100000, seed=rand.randint(0,2**31-1),
                    cooling_schedule = paper_cooling_schedule,
                    step_size_schedule = sqrt_step_size_schedule,
                    random_step_likelihood=p
                    )
        sim.run_simulation(10000)
        arrs[i,j] = sim.E

    print('.', end='')

    
np.save('data/data-4-{}-{}.npy'.format(num_particles, num_probs), arrs)





In [ ]:

arrs = np.load('data/data-4-50-50.npy')
arrs = arrs[2:]
num_probs = 20
probs = np.linspace(0,1,num_probs)
probs1 = probs[2:]
print(arrs.shape)
mean = np.mean(arrs, axis=1)
min_energy = np.min(arrs)
plt.plot(probs1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
plt.fill_between(probs1, np.percentile(arrs, 0, axis=1), np.percentile(arrs, 99, axis=1), alpha=0.3)
plt.plot(probs1, min_energy*np.ones_like(probs1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('P(random step)')
plt.ylabel('E')
plt.legend()